## **Date Formatter — JavaScript solution + full thought process**

### **Thought procedure — how to approach the problem**

1. **Read the spec carefully.**
   — Input: full English month name, space, day, comma, space (may vary), 4-digit year.
   — Output: ISO-style `YYYY-MM-DD` with zero-padded month and day.

2. **Decide parsing method.**
   Use a small parser (regex + mapping) rather than `Date.parse` because month names and formats are predictable and you want consistent output and validation.

3. **Break into parts.**
   Extract month name, day number, and year. Normalize (trim, consistent case) so `"january"` / extra spaces won't break things.

4. **Map month name → month number.**
   Use an object that maps `"January"` → `1`, etc. This avoids locale surprises.

5. **Validate.**

   * Check the regex matched.
   * Check the month name exists.
   * Check day is within valid range for that month (account for leap years).
   * Check year is sensible (e.g. 4 digits).

6. **Format output.**

   * Zero pad month and day to two digits (e.g. `6` → `06`) using `String.prototype.padStart`.
   * Return `${year}-${MM}-${DD}`.

7. **Edge cases to consider.**

   * Invalid month name → throw or return error.
   * Day out of range (e.g., `"February 30, 2021"`) → error.
   * Extra spaces or mixed-case input → handle by trimming and normalizing case.
   * Years outside 4 digits — decide policy (here we expect 4+ digits but will accept >=1 digit).

---

# Implementation

In [ ]:
/**
 * Convert "Month day, year" -> "YYYY-MM-DD"
 * Example: "December 6, 2025" -> "2025-12-06"
 *
 * Throws an Error on invalid input.
 */
function formatDate(input) {
  if (typeof input !== "string") {
    throw new Error('Input must be a string in the format "Month day, year".');
  }

  // 1) Trim and normalize spacing
  const s = input.trim();

  // 2) Regex to extract three parts: monthName, day, year
  // Accepts variable spaces, case-insensitive month names
  const match = s.match(/^([A-Za-z]+)\s+(\d{1,2}),\s*(\d{1,})$/);
  if (!match) {
    throw new Error(
      'Invalid date format. Expected "Month day, year" (e.g. "December 6, 2025").'
    );
  }

  let [, monthName, dayStr, yearStr] = match;
  monthName = monthName.toLowerCase();

  // 3) Map month names to numbers (1-12)
  const months = {
    january: 1,
    february: 2,
    march: 3,
    april: 4,
    may: 5,
    june: 6,
    july: 7,
    august: 8,
    september: 9,
    october: 10,
    november: 11,
    december: 12,
  };

  const monthNum = months[monthName];
  if (!monthNum) {
    throw new Error(`Invalid month name: "${monthName}".`);
  }

  // 4) Parse day and year
  const day = parseInt(dayStr, 10);
  const year = parseInt(yearStr, 10);

  if (!Number.isFinite(day) || !Number.isFinite(year) || day < 1) {
    throw new Error("Invalid day or year.");
  }

  // 5) Validate day is within range for the month, considering leap years
  // Use Date to find days in month reliably:
  const daysInMonth = new Date(year, monthNum, 0).getDate(); // monthNum is 1-12, new Date(year, monthNum, 0) -> last day of monthNum
  if (day > daysInMonth) {
    throw new Error(
      `${
        monthName[0].toUpperCase() + monthName.slice(1)
      } ${day}, ${year} is not a valid date.`
    );
  }

  // 6) Zero-pad month and day to two digits
  const mm = String(monthNum).padStart(2, "0");
  const dd = String(day).padStart(2, "0");

  return `${year}-${mm}-${dd}`;
}


# Examples / Tests

In [ ]:
console.log(formatDate("December 6, 2025"));    // "2025-12-06"
console.log(formatDate("January 1, 2020"));     // "2020-01-01"
console.log(formatDate(" february 29, 2020 ")); // "2020-02-29"  (leap year)
console.log(formatDate("February 29, 2021"));   // throws Error (invalid date)
console.log(formatDate("April 31, 2022"));      // throws Error (April has 30 days)

# Complexity & notes

* **Time complexity:** O(1) — constant time string operations and small map lookup.
* **Space complexity:** O(1).
* **Why not use `Date.parse` directly?**
  `Date.parse`/`new Date(string)` behavior depends on implementation and locale; parsing manually with an explicit month map gives predictable output and easier validation.
* **Extensibility:**
  If you want to accept abbreviated months (`"Dec"`), multilingual month names, or different year lengths, extend the `months` map and adjust the regex.

---

# Final quick checklist when you face similar tasks

1. Identify exact input format and allowable variations.
2. Break input into parts (tokenize).
3. Normalize (trim, case).
4. Convert/validate each part.
5. Reassemble in desired format, ensuring padding/zeroes.
6. Add clear error handling for invalid inputs

Below is a **super clear, line-by-line explanation** of every part of the code.
I will break it into sections so you understand the *why* behind every step — not just the *what*.

---

# ✅ The Code (for reference)

In [ ]:
function formatDate(input) {
  if (typeof input !== "string") {
    throw new Error('Input must be a string in the format "Month day, year".');
  }

  const s = input.trim();

  const match = s.match(/^([A-Za-z]+)\s+(\d{1,2}),\s*(\d{1,})$/);
  if (!match) {
    throw new Error(
      'Invalid date format. Expected "Month day, year" (e.g. "December 6, 2025").'
    );
  }

  let [, monthName, dayStr, yearStr] = match;
  monthName = monthName.toLowerCase();

  const months = {
    january: 1,
    february: 2,
    march: 3,
    april: 4,
    may: 5,
    june: 6,
    july: 7,
    august: 8,
    september: 9,
    october: 10,
    november: 11,
    december: 12,
  };

  const monthNum = months[monthName];
  if (!monthNum) {
    throw new Error(`Invalid month name: "${monthName}".`);
  }

  const day = parseInt(dayStr, 10);
  const year = parseInt(yearStr, 10);

  if (!Number.isFinite(day) || !Number.isFinite(year) || day < 1) {
    throw new Error("Invalid day or year.");
  }

  const daysInMonth = new Date(year, monthNum, 0).getDate();
  if (day > daysInMonth) {
    throw new Error(`${monthName} ${day}, ${year} is not a valid date.`);
  }

  const mm = String(monthNum).padStart(2, "0");
  const dd = String(day).padStart(2, "0");

  return `${year}-${mm}-${dd}`;
}


# 🔍 FULL EXPLANATION — **EVERY LINE & WHY IT’S THERE**

---

## **1. Function definition**

```javascript
function formatDate(input) {
```

You create a function named **formatDate** that takes one argument:
`input` → the string like `"December 6, 2025"`.

---

## **2. Validate the input type**

```javascript
if (typeof input !== 'string') {
  throw new Error('Input must be a string in the format "Month day, year".');
}
```

Why?

* You must ensure the function receives a valid **string**.
* If someone passes a number, boolean, array, etc., the rest of the code would break.
* So you protect the function with an early error.

---

## **3. Remove extra spaces**

```javascript
const s = input.trim();
```

Why?

* User might pass `" December 6, 2025 "` with spaces.
* `trim()` removes whitespace from both ends.

Example:
`" December 6, 2025 "` → `"December 6, 2025"`

---

## **4. Use regex to extract Month, Day, Year**

```javascript
const match = s.match(/^([A-Za-z]+)\s+(\d{1,2}),\s*(\d{1,})$/);
```

This is **the heart of the parser**.

### The REGEX explained:

```
^                 → start of string  
([A-Za-z]+)       → month name (letters only)  
\s+               → one or more spaces  
(\d{1,2})         → day (1 or 2 digits)  
,                 → literal comma  
\s*               → zero or more spaces  
(\d{1,})          → year (1 or more digits)  
$                 → end of string
```

Example match groups:

Input: `"December 6, 2025"`

| Group | Value      | Meaning    |
| ----- | ---------- | ---------- |
| 1     | "December" | month name |
| 2     | "6"        | day        |
| 3     | "2025"     | year       |

---

## **5. If regex fails, throw error**

```javascript
if (!match) {
  throw new Error('Invalid date format...');
}
```

Why?

* If the input doesn’t match the expected structure, the program must fail early.

Examples that FAIL:

* `"Dec 6 2025"`
* `"December 6 2025"`
* `"December, 6 2025"`
* `"Random text"`

---

## **6. Extract the values from the regex match**

```javascript
let [, monthName, dayStr, yearStr] = match;
```

The first element (`match[0]`) is the full string.
We skip it using a leading comma.

Extracted:

* `monthName` → `"December"`
* `dayStr` → `"6"`
* `yearStr` → `"2025"`

---

## **7. Normalize the month name**

```javascript
monthName = monthName.toLowerCase();
```

Why?

* Easier to match: `"December"` → `"december"`
* Makes it case-insensitive: `"jANuArY"` → `"january"`

---

## **8. Month name → month number lookup table**

```javascript
const months = {
  january: 1, february: 2, march: 3, april: 4,
  may: 5, june: 6, july: 7, august: 8,
  september: 9, october: 10, november: 11, december: 12
};
```

Why?

* JavaScript does not automatically convert month names.
* So you manually map month strings to month numbers.

---

## **9. Lookup the month number**

```javascript
const monthNum = months[monthName];
```

If `monthName = "december"`
then `monthNum = 12`.

---

## **10. Validate month name**

```javascript
if (!monthNum) {
  throw new Error(`Invalid month name: "${monthName}".`);
}
```

If someone passes `"Decembrr"` → undefined → throw.

---

## **11. Convert day & year from string → number**

```javascript
const day = parseInt(dayStr, 10);
const year = parseInt(yearStr, 10);
```

Why?

* Regex extracted them as strings.
* You need numeric values for validation.

Example:

* `"6"` → `6`
* `"2025"` → `2025`

---

## **12. Validate day/year basic logic**

```javascript
if (!Number.isFinite(day) || !Number.isFinite(year) || day < 1) {
  throw new Error('Invalid day or year.');
}
```

Checks:

* day & year must be valid numbers
* day can’t be 0 or negative

---

## **13. Validate the date actually exists**

```javascript
const daysInMonth = new Date(year, monthNum, 0).getDate();
```

⚡ Trick: `new Date(year, monthNum, 0)`
gives **the last day of the previous month**.

Since monthNum is *already 1–12*:

* For February (2), `new Date(year, 2, 0)` → Feb 28 or Feb 29 (leap year)
* For April (4), → April 30

Why this works?

* JavaScript auto-adjusts dates.

---

## **14. Check day is within valid range**

```javascript
if (day > daysInMonth) {
  throw new Error(`${monthName} ${day}, ${year} is not a valid date.`);
}
```

Examples that fail:

* `"February 30, 2021"`
* `"April 31, 2022"`
* `"September 31, 2025"`

---

## **15. Zero-pad the month**

```javascript
const mm = String(monthNum).padStart(2, '0');
```

Examples:

* `12` → `"12"`
* `3` → `"03"`

---

## **16. Zero-pad the day**

```javascript
const dd = String(day).padStart(2, '0');
```

Examples:

* `6` → `"06"`
* `25` → `"25"`

---

## **17. Construct final ISO date format**

```javascript
return `${year}-${mm}-${dd}`;
```

Output for `"December 6, 2025"`:

```
"2025-12-06"
```




# ⭐ Final Summary — What the code does

1. Validates the input is a string
2. Removes extra spaces
3. Uses regex to extract month/day/year
4. Converts month name → number
5. Parses numbers
6. Validates date logically
7. Zero-pads month/day
8. Returns final `"YYYY-MM-DD"` format